# Marktstammdaten 

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sqlite3
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.geocoders import Nominatim
import datetime, time
from decimal import Decimal

## Marktstammdaten einlesen

In [2]:
# Marktstammdaten-Tabelle einlesen
marktstammdaten = pd.read_pickle('marktstammdaten_ags_plz.pkl')
marktstammdaten

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14 07:21:05.450956000,1,ABR941126007873,Nordrhein-Westfalen,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,...,Kein Stromspeicher vorhanden,40,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Ost,< 20 Grad,Haushalt,05970,2009
3173006,2020-04-29 09:37:03.402485600,0,ABR903375397990,Nordrhein-Westfalen,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,...,Kein Stromspeicher vorhanden,98,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,40 - 60 Grad,Landwirtschaft,05958,2010
3173007,2020-11-26 08:57:11.006049400,1,ABR985070432688,Bayern,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,...,Kein Stromspeicher vorhanden,90,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",0.0,West,20 - 40 Grad,Haushalt,09374,2020
3173008,2020-05-02 10:16:32.781845900,1,ABR925833676461,Hessen,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,...,Stromspeicher vorhanden - kein gemeinsamer Wec...,15,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,06433,2016


In [3]:
#marktstammdaten.isnull().sum()

In [4]:
marktstd = marktstammdaten.copy()
marktstd.head()

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011


## Geographische Koordinaten zur PLZ ermitteln  
Aus Ort und Postleitzahl sollen **geogr. Breite** und **geogr. Laenge** ermittelt werden, um die Standorte der PV-Anlagen    
später mit der jeweils nächstgelegenen Wetterstation zu korrellieren.   
Source: https://thepythoncode.com/article/get-geolocation-in-python

In [5]:
# Geolocation Tabelle erstellen
geolocation = marktstd[['Postleitzahl']].drop_duplicates()
geolocation.reset_index(drop=True, inplace=True)
geolocation

,Postleitzahl
0,48147
1,73529
2,14641
3,93080
4,66740
...,...
8155,33333
8156,12687
8157,25999
8158,10779


In [6]:
# Ermittlung der Koordinaten (Länge und Breite) für alle Postleizahlen 

#geolocator = Nominatim(user_agent='myApp')
#
#for index, row in geolocation.iterrows():
#    postleitzahl = row['Postleitzahl']
#    
#    try:
#        location = geolocator.geocode(postleitzahl + ', Germany')
#        if location:
#            geolocation.at[index, 'geogr. Breite'] = location.latitude
#            geolocation.at[index, 'geogr. Laenge'] = location.longitude
#    except Exception as e:
#        print(f"Fehler bei Postleitzahl {postleitzahl}: {str(e)}")
#geolocation

**HINWEIS:**   
Die Geolocator- Funktion wurde auskommentiert, da sie sehr lange Zeit in Anspruch nimmt.    
Die gewonnenen Geodaten wurden als Pickle gespeichert, um sie im Notebook weiterzuverwenden.    

In [7]:
# Geolocation als Pickle speichern
#geolocation_raw = geolocation.copy()
#geolocation_raw.to_pickle('geolocation_plz_koordinaten.pkl')

In [8]:
# Geolocation-Tabelle einlesen
geoloc_plz = pd.read_pickle('geolocation_plz_koordinaten.pkl')
geoloc_plz

,Postleitzahl,geogr. Breite,geogr. Laenge
0,48147,51.974960,7.635058
1,73529,48.779440,9.836111
2,14641,52.578151,12.892790
3,93080,48.977788,12.057678
4,66740,49.319471,6.753036
...,...,...,...
8155,33333,51.883691,8.443135
8156,12687,52.556674,13.565015
8157,25999,54.955273,8.344864
8158,10779,52.492014,13.339645


In [9]:
geoloc_plz.isna().sum()

Postleitzahl     0
geogr. Breite    2
geogr. Laenge    2
dtype: int64

In [10]:
missing = geoloc_plz[geoloc_plz.isna().any(axis=1)]
missing

,Postleitzahl,geogr. Breite,geogr. Laenge
1363,88400,NaN,NaN
4933,73265,NaN,NaN


Für 2 Postleitzahlen wurde keine Geolocation zugeordnet.   
Daher werden die fehlenden Werte von Hand ersetzt.    

Postleitzahl **88400** gehört zu Biberach.    
Breitengrad: 50° 16' 48'' N (Dezimalgrad: 48.09297)   
Längengrad: 9° 53' 46'' O (Dezimalgrad: 9.78623)    
     
Postleitzahl **73265** gehört zu Dettingen.     
Breitengrad: 48° 37' 6'' N (Dezimalgrad: 48.61566)   
Längengrad: 9° 26' 42'' O (Dezimalgrad: 9.45560)     
      
Quelle:   
https://www.geoplaner.de/   
https://www.wogibtes.info/plz/D-88400/   

In [11]:
# fehlende Werte für PLZ 97779 ergänzen
geoloc_plz.at[1363, 'geogr. Breite'] = 48.09297
geoloc_plz.at[1363, 'geogr. Laenge'] = 9.78623
geoloc_plz.at[4933, 'geogr. Breite'] = 48.61566
geoloc_plz.at[4933, 'geogr. Laenge'] = 9.45560 
geoloc_plz

,Postleitzahl,geogr. Breite,geogr. Laenge
0,48147,51.974960,7.635058
1,73529,48.779440,9.836111
2,14641,52.578151,12.892790
3,93080,48.977788,12.057678
4,66740,49.319471,6.753036
...,...,...,...
8155,33333,51.883691,8.443135
8156,12687,52.556674,13.565015
8157,25999,54.955273,8.344864
8158,10779,52.492014,13.339645


In [12]:
geoloc_plz.isna().sum()

Postleitzahl     0
geogr. Breite    0
geogr. Laenge    0
dtype: int64

In [13]:
# Merge Geolocation mit Marktstammdaten
#marktstd= marktstd.merge(geoloc_plz, on="Postleitzahl", how="left")
#marktstd

In [14]:
#marktstd.isna().sum()

# Wetter- Daten

## Sonnenstunden
Bezugszeitraum: 1991-2020   
Quelle: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/multi_annual/mean_91-20/
Dateien: Sonnenscheindauer_1991-2020.txt    
         Sonnenscheindauer_1991-2020_Stationsliste.txt   

In [15]:
# Sonnenstunden pro Wetterstation einlesen
columns = ['Stations_id', 'Jahr']
sonnenstunden_raw = pd.read_csv("../input/Data/Sonnenscheindauer_1991-2020.txt", sep=";", encoding='latin-1',
                               usecols=columns)
sonnenstunden_raw = sonnenstunden_raw.rename(columns={'Jahr' : 'Sonnenstunden pro Jahr'})
sonnenstunden_raw

,Stations_id,Sonnenstunden pro Jahr
0,1,1735.9
1,3,1653.3
2,44,1571.0
3,52,1602.2
4,61,1770.6
...,...,...
1086,19172,1646.9
1087,19607,1623.8
1088,19617,1671.5
1089,19647,1686.4


In [16]:
# Daten der Wetterstationen einlesen
columns = ['Stations_id', 'Stationsname', 'geogr. Breite','geogr. Laenge']
wetterstation = pd.read_csv("../input/Data/Sonnenscheindauer_1991-2020_Stationsliste.txt", sep=";", encoding='latin-1',
                              usecols=columns)
wetterstation

,Stations_id,Stationsname,geogr. Breite,geogr. Laenge
0,1,Aach,47.841299,8.849299
1,3,Aachen,50.782659,6.094087
2,44,Großenkneten,52.933593,8.237018
3,52,Ahrensburg-Wulfsdorf,53.662339,10.199036
4,61,Aiterhofen,48.844306,12.617053
...,...,...,...,...
1083,19172,Wacken,54.024600,9.387966
1084,19607,Cämmerswalde-Rauschenbach,50.695202,13.497187
1085,19617,Burghaslach,49.739130,10.603913
1086,19647,Eberbach/Neckar,49.454703,8.979441


In [17]:
# Mergen der beiden Tabellen
sonnenstunden_raw= sonnenstunden_raw.merge(wetterstation, on='Stations_id', how='left')
sonnenstunden_raw

,Stations_id,Sonnenstunden pro Jahr,Stationsname,geogr. Breite,geogr. Laenge
0,1,1735.9,Aach,47.841299,8.849299
1,3,1653.3,Aachen,50.782659,6.094087
2,44,1571.0,Großenkneten,52.933593,8.237018
3,52,1602.2,Ahrensburg-Wulfsdorf,53.662339,10.199036
4,61,1770.6,Aiterhofen,48.844306,12.617053
...,...,...,...,...,...
1086,19172,1646.9,Wacken,54.024600,9.387966
1087,19607,1623.8,Cämmerswalde-Rauschenbach,50.695202,13.497187
1088,19617,1671.5,Burghaslach,49.739130,10.603913
1089,19647,1686.4,Eberbach/Neckar,49.454703,8.979441


### Data Cleaning

In [18]:
sonnenstunden_raw.isna().sum()

Stations_id               0
Sonnenstunden pro Jahr    0
Stationsname              3
geogr. Breite             3
geogr. Laenge             3
dtype: int64

In [19]:
missingss = sonnenstunden_raw[sonnenstunden_raw.isna().any(axis=1)]
missingss

,Stations_id,Sonnenstunden pro Jahr,Stationsname,geogr. Breite,geogr. Laenge
118,601,1602.9,NaN,NaN,NaN
486,2843,1694.5,NaN,NaN,NaN
1071,14138,1779.0,NaN,NaN,NaN


In [20]:
# Für drei der Mess-Stationen liegen weder Name noch Angaben zur Lage vor.   
# Löschen der Zeilen mit fehlenden Daten.
sonnenstunden_raw.dropna(inplace=True)
sonnenstunden_raw.reset_index()

,index,Stations_id,Sonnenstunden pro Jahr,Stationsname,geogr. Breite,geogr. Laenge
0,0,1,1735.9,Aach,47.841299,8.849299
1,1,3,1653.3,Aachen,50.782659,6.094087
2,2,44,1571.0,Großenkneten,52.933593,8.237018
3,3,52,1602.2,Ahrensburg-Wulfsdorf,53.662339,10.199036
4,4,61,1770.6,Aiterhofen,48.844306,12.617053
...,...,...,...,...,...,...
1083,1086,19172,1646.9,Wacken,54.024600,9.387966
1084,1087,19607,1623.8,Cämmerswalde-Rauschenbach,50.695202,13.497187
1085,1088,19617,1671.5,Burghaslach,49.739130,10.603913
1086,1089,19647,1686.4,Eberbach/Neckar,49.454703,8.979441


### Zuordnung der PV-Anlagen zur nächstgelegenen Wetterstationen
Methode: Haversine’s Equation   
Quelle: https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546   

In [21]:
# Umbenennen der Spalten
sonnenstunden = sonnenstunden_raw.rename(columns = {'geogr. Laenge':'lat','geogr. Breite':'lon'})
sun_plz = geoloc_plz.rename(columns = {'geogr. Laenge':'lat','geogr. Breite':'lon'})

In [22]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [23]:
def find_nearest(lat, long):
    distances = sonnenstunden.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    return sonnenstunden.loc[distances.idxmin(), 'Sonnenstunden pro Jahr']

In [24]:
sun_plz['Sonnenstunden'] = sun_plz.apply(
    lambda row: find_nearest(row['lat'], row['lon']), axis=1)
sun_plz

,Postleitzahl,lon,lat,Sonnenstunden
0,48147,51.974960,7.635058,1582.4
1,73529,48.779440,9.836111,1717.6
2,14641,52.578151,12.892790,1698.4
3,93080,48.977788,12.057678,1681.2
4,66740,49.319471,6.753036,1716.0
...,...,...,...,...
8155,33333,51.883691,8.443135,1571.0
8156,12687,52.556674,13.565015,1752.2
8157,25999,54.955273,8.344864,1717.9
8158,10779,52.492014,13.339645,1812.4


In [25]:
sun_plz.isna().sum()

Postleitzahl     0
lon              0
lat              0
Sonnenstunden    0
dtype: int64

In [26]:
#Tabelle Sonnenstunden nach PLZ erstellen
sonnenstunden_plz = sun_plz[['Postleitzahl', 'Sonnenstunden']].copy()
sonnenstunden_plz

,Postleitzahl,Sonnenstunden
0,48147,1582.4
1,73529,1717.6
2,14641,1698.4
3,93080,1681.2
4,66740,1716.0
...,...,...
8155,33333,1571.0
8156,12687,1752.2
8157,25999,1717.9
8158,10779,1812.4


# Photovoltaik Ausbau und Potenziale

## Anteil Photovoltaik an der Brutto-Stromerzeugung in Dtl.
Zeitraum: 2002-2022  
Quelle: https://de.statista.com/statistik/daten/studie/250915/umfrage/anteil-der-photovoltaik-an-der-stromerzeugung-in-deutschland/

In [27]:
pv_anteil_raw = pd.read_excel("../input/Data/statistic_id250915_photovoltaik---anteil-an-der-stromerzeugung-in-deutschland-bis-2022.xlsx", sheet_name="Daten", index_col=0)
#v_anteil_raw

In [28]:
# Reset Index, Spalten umbenennen
pv_anteil_raw.reset_index(drop=False, inplace=True)
pv_anteil_raw = pv_anteil_raw.rename(columns={'index': 'Year'})
pv_anteil_raw

,Jahr,% Anteil
0,2002,0.0
1,2003,0.1
2,2004,0.1
3,2005,0.2
4,2006,0.3
5,2007,0.5
6,2008,0.7
7,2009,1.1
8,2010,1.8
9,2011,3.2


# Strom-Verbraucherpreise
Strom Verbraucherpreise für private Haushalte (in ct pro kWh inkl. MwSt)   
Zeitraum: 2000 bis 2023   
Quellen:   
https://www.bdew.de/presse/pressemappen/strompreis/   
https://www.bmwk.de/Redaktion/DE/Binaer/Energiedaten/energiedaten-gesamt-xls-2022.pdf?__blob=publicationFile&v=3    


In [29]:
strompreis_raw = pd.read_excel("../input/Data/strompreise_einspeisevergütung.xlsx", sheet_name="Strompreise", index_col=0)
strompreis_raw

,Strom Verbraucherpreise für private Haushalte (in ct pro kWh inkl. MwSt)
Jahr,
2000,14.92
2001,15.44
2002,16.08
2003,16.86
2004,17.51
2005,18.23
2006,18.91
2007,20.15
2008,21.43


In [30]:
# Reset Index, Spalte umbenennen
strompreis_raw.reset_index(drop=False, inplace=True)
strompreis = strompreis_raw.rename(columns={'Strom Verbraucherpreise für private Haushalte (in ct pro kWh inkl. MwSt)': 'Strom_Verbraucherpreis'})
strompreis

,Jahr,Strom_Verbraucherpreis
0,2000,14.92
1,2001,15.44
2,2002,16.08
3,2003,16.86
4,2004,17.51
5,2005,18.23
6,2006,18.91
7,2007,20.15
8,2008,21.43
9,2009,22.72


# EEG Einspeisevergütung
Quellen:   
https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/ErneuerbareEnergien/EEG_Foerderung/Archiv_VergSaetze/start.html   
https://www.sfv.de/lokal/mails/sj/verguetu    
https://www.lfl.bayern.de/iba/energie/161645/index.php   
     
Bis 2012 wurde die Einspeisevergütung für kleine, in privaten Haushalten gebräuchlichen PV-Anlagen bis 30kW Leistung in einer Maßzahl festgelegt. Seit April 2012 wird zwischen Anlagen bis 10kW und Anlagen über 10 bis 40kW unterschieden. Um die durchschnittliche Entwicklung der Einspeisevergütung für kleine Anlagen zu beurteilen, wurde für die Werte ab 04/2012 der Mittelwert gebildet.


In [31]:
einspeisung_raw = pd.read_excel("../input/Data/strompreise_einspeisevergütung.xlsx", sheet_name="Einspeiseverguetung", index_col=0)
einspeisung_raw

,"EEG Einspeisevergütung für kleine Anlagen bis 30kW (in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)","EEG Einspeisevergütung für kleine Anlagen ≤10kW(in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)","EEG Einspeisevergütung für kleine Anlagen >10-40kW (in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)",Durchschnittliche Einspeisevergütung für kleine bis mittlere Anlagen
Datum,,,,
2000-01-01,50.62,NaN,NaN,50.62
2001-01-01,50.62,NaN,NaN,50.62
2002-01-01,48.10,NaN,NaN,48.10
2003-01-01,45.70,NaN,NaN,45.70
2004-01-01,57.40,NaN,NaN,57.40
...,...,...,...,...
2023-08-01,NaN,8.2,7.1,7.65
2023-09-01,NaN,8.2,7.1,7.65
2023-10-01,NaN,8.2,7.1,7.65


In [32]:
# Reset Index, Spalte umbenennen
einspeisung_raw.reset_index(drop=False, inplace=True)
einspeisung_raw = einspeisung_raw.rename(columns={'Durchschnittliche Einspeisevergütung für kleine bis mittlere Anlagen': 'Einspeiseverguetung_kleine_PV_Anlagen'})
einspeisung_raw

,Datum,"EEG Einspeisevergütung für kleine Anlagen bis 30kW (in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)","EEG Einspeisevergütung für kleine Anlagen ≤10kW(in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)","EEG Einspeisevergütung für kleine Anlagen >10-40kW (in Ct / kWh, gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)",Einspeiseverguetung_kleine_PV_Anlagen
0,2000-01-01,50.62,NaN,NaN,50.62
1,2001-01-01,50.62,NaN,NaN,50.62
2,2002-01-01,48.10,NaN,NaN,48.10
3,2003-01-01,45.70,NaN,NaN,45.70
4,2004-01-01,57.40,NaN,NaN,57.40
...,...,...,...,...,...
149,2023-08-01,NaN,8.2,7.1,7.65
150,2023-09-01,NaN,8.2,7.1,7.65
151,2023-10-01,NaN,8.2,7.1,7.65
152,2023-11-01,NaN,8.2,7.1,7.65


In [33]:
# Spalten löschen
drop_cols = ['EEG Einspeisevergütung für kleine Anlagen bis 30kW (in Ct / kWh,  gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)',
       'EEG Einspeisevergütung für kleine Anlagen ≤10kW(in Ct / kWh,  gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)',
       'EEG Einspeisevergütung für kleine Anlagen >10-40kW (in Ct / kWh,  gültig für 20 Jahre plus Restmonate des Inbetriebnahmejahres)']            
einspeisung = einspeisung_raw.drop(drop_cols, axis=1)
einspeisung

,Datum,Einspeiseverguetung_kleine_PV_Anlagen
0,2000-01-01,50.62
1,2001-01-01,50.62
2,2002-01-01,48.10
3,2003-01-01,45.70
4,2004-01-01,57.40
...,...,...
149,2023-08-01,7.65
150,2023-09-01,7.65
151,2023-10-01,7.65
152,2023-11-01,7.65


In [34]:
print(einspeisung.columns)

Index(['Datum', 'Einspeiseverguetung_kleine_PV_Anlagen'], dtype='object')


In [35]:
einspeisung.set_index('Datum', inplace=True)

# Neuer Index mit allen Monaten zwischen Januar 2000 und Dezember 2023
date_range = pd.date_range(start='2000-01-01', end='2023-12-01', freq='MS')

# Auffüllen von fehlenden Werten mit vorherigem Wert
einspeisung = einspeisung.reindex(date_range)
einspeisung.fillna(method='ffill', inplace=True)

einspeisung.reset_index(inplace=True)
einspeisung.columns = ['Datum', 'Einspeiseverguetung_kleine_PV_Anlagen']

einspeisung

,Datum,Einspeiseverguetung_kleine_PV_Anlagen
0,2000-01-01,50.62
1,2000-02-01,50.62
2,2000-03-01,50.62
3,2000-04-01,50.62
4,2000-05-01,50.62
...,...,...
283,2023-08-01,7.65
284,2023-09-01,7.65
285,2023-10-01,7.65
286,2023-11-01,7.65


# EEG-Umlage
Quellen:   
https://de.statista.com/statistik/daten/studie/152973/umfrage/eeg-umlage-entwicklung-der-strompreise-in-deutschland-seit-2000/    
https://www.enercity.de/magazin/unsere-welt/was-ist-die-eeg-umlage    
https://www.rewag.de/aktuelle-marktlage/eeg-umlage#group3578   

Die EEG-Umlage dient der Förderung erneuerbarer Energien nach dem Erneuerbare-Energien-Gesetz. Sie ist Teil des Strompreises und stellt die Differenzkosten zwischen dem Marktpreis für Strom und dem Fördersatz für erneuerbare Energieträger dar. 

In [36]:
eeg_umlage_raw = pd.read_excel("../input/Data/eeg-umlage-fuer-haushaltsstromkunden.xlsx", sheet_name="Daten", index_col=0)
eeg_umlage_raw.reset_index(inplace=True)
eeg_umlage_raw

,Datum,Euro-Cent pro kWh
0,2000-01-01,0.2
1,2000-02-01,0.2
2,2000-03-01,0.2
3,2000-04-01,0.2
4,2000-05-01,0.2
...,...,...
283,2023-08-01,0.0
284,2023-09-01,0.0
285,2023-10-01,0.0
286,2023-11-01,0.0


# PV-Kosten pro kWp   
Quellen:   
https://www.solarwatt.de/ratgeber/kwh-kwp   
https://gruenes.haus/photovoltaik-preisentwicklung/   
https://www.tech-for-future.de/preisentwicklung-photovoltaik/   
https://ourworldindata.org/grapher/solar-pv-prices   

Mit kWp bezeichnet man die theoretisch größtmögliche Leistung, die eine Photovoltaikanlage unter bestimmten Bedingungen erreichen kann. Damit ist die Einheit kWp von Bedeutung, wenn es darum geht, die Leistung von Anlagen, aber auch einzelnen Solarmodulen vergleichbar zu machen.   

Die Maßeinheit Kilowatt-Peak bzw. Watt-Peak wurde geschaffen, weil Photovoltaikanlagen abhängig von den Umgebungsbedingungen sehr unterschiedliche Stromerträge produzieren. Ausschlaggebend sind die Sonneneinstrahlung, der Einfallswinkel sowie die Temperatur der Solarzellen. Die sogenannte Nennleistung von Solarmodulen, gemessen in Kilowatt-Peak, wird daher unter immer gleichbleibenden, genormten Testbedingungen ermittelt.   

In [37]:
pv_kosten_raw = pd.read_excel("../input/Data/preise_solarmodule_pro_jahr.xlsx", sheet_name="pv_daten", index_col=0)
pv_kosten_raw.reset_index(inplace=True)
pv_kosten_raw

,Jahr_Monat,PV_Kosten_Euro_pro_kWp
0,2000-01-01,8100
1,2000-02-01,8100
2,2000-03-01,8100
3,2000-04-01,8100
4,2000-05-01,8100
...,...,...
283,2023-08-01,1250
284,2023-09-01,1250
285,2023-10-01,1250
286,2023-11-01,1250


# Relevante historische Ereignisse   
- Solardeckel  
- Energie-Krise, ausgelöst durch den Ukraine-Krieg und Sanktionen gegen Russland  
- Nuklear-Katastrophe in Fukushima 

Der Effekt des jeweiligen Ereignisses wurde mit 0 (keine Wirkung) oder 1 (Auswirkungen) kodiert.  

Quellen:    
https://www.energie-chronik.de/yeeg.htm    
https://www.energie-chronik.de/ypvstrom.htm   
https://www.bayern-innovativ.de/de/seite/auswirkungen-nuklearkatastrophe-fukushima     
https://www.lpb-bw.de/fukushima    
https://www.wirtschaftsdienst.eu/inhalt/jahr/2023/heft/11/beitrag/energiekrise-belastet-haushalte.html   
https://www.consilium.europa.eu/de/policies/energy-prices-and-security-of-supply/   

In [38]:
ereignisse_raw = pd.read_excel("../input/Data/ereignisse.xlsx", sheet_name="Data", index_col=0)
ereignisse_raw.reset_index(inplace=True)
ereignisse_raw

,Datum,Fukushima,Ukrainekrieg,Solardeckel
0,2000-01-01,0,0,0
1,2000-02-01,0,0,0
2,2000-03-01,0,0,0
3,2000-04-01,0,0,0
4,2000-05-01,0,0,0
...,...,...,...,...
283,2023-08-01,0,0,0
284,2023-09-01,0,0,0
285,2023-10-01,0,0,0
286,2023-11-01,0,0,0


# E-Mobility und erneuerbare Energien

## E-Autos und Ladestationen
Gebietsstand zum 31.12.2021   
Tabelle: Deutschlandatlas_KRS1221.csv   
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html   

**eAuto:** Anteil von Pkw mit reinem Elektroantrieb an allen Pkw zum 01.01.2022 in %   
**eLade:** Öffentlich zugängliche Ladepunkte für E-Autos zum 01.10.2022 je 100.000 Einwohner   

In [39]:
# Datensatz einlesen
columns = ["Gebietskennziffer", "name", "eauto", "elade"]
eauto_raw = pd.read_csv("../input/Deutschlandatlas/Deutschlandatlas_KRS1221.csv", usecols=columns, encoding="latin1", 
                             dtype={"Gebietskennziffer" : str}, sep=";", decimal=",", low_memory=False)
eauto_raw = eauto_raw.rename(columns={'elade': 'eLade', 'eauto': 'eAuto', 'name' : 'Ort'})
eauto_raw

,Gebietskennziffer,Ort,eLade,eAuto
0,1001000,"Flensburg, Stadt",117.44,1.63
1,1002000,"Kiel, Landeshauptstadt",99.50,1.37
2,1003000,"Lübeck, Hansestadt",61.96,1.15
3,1004000,"Neumünster, Stadt",56.61,1.60
4,1051000,Dithmarschen,158.25,1.19
...,...,...,...,...
395,16073000,Saalfeld-Rudolstadt,62.40,0.53
396,16074000,Saale-Holzland-Kreis,122.40,0.69
397,16075000,Saale-Orla-Kreis,94.90,0.49
398,16076000,Greiz,32.26,0.49


### Data Cleaning

In [40]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
eauto_raw['Gebietskennziffer'] = eauto_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
eauto_raw['AGS-5'] = eauto_raw['Gebietskennziffer'].str[:-3]
eauto_raw
# Quelle: ChatGPT (siehe Protokoll ChatGPT, Seite 3)

,Gebietskennziffer,Ort,eLade,eAuto,AGS-5
0,01001000,"Flensburg, Stadt",117.44,1.63,01001
1,01002000,"Kiel, Landeshauptstadt",99.50,1.37,01002
2,01003000,"Lübeck, Hansestadt",61.96,1.15,01003
3,01004000,"Neumünster, Stadt",56.61,1.60,01004
4,01051000,Dithmarschen,158.25,1.19,01051
...,...,...,...,...,...
395,16073000,Saalfeld-Rudolstadt,62.40,0.53,16073
396,16074000,Saale-Holzland-Kreis,122.40,0.69,16074
397,16075000,Saale-Orla-Kreis,94.90,0.49,16075
398,16076000,Greiz,32.26,0.49,16076


In [41]:
# Spalten löschen
drop_cols = ['Gebietskennziffer', 'Ort']            
eauto_raw = eauto_raw.drop(drop_cols, axis=1)
eauto_raw

,eLade,eAuto,AGS-5
0,117.44,1.63,01001
1,99.50,1.37,01002
2,61.96,1.15,01003
3,56.61,1.60,01004
4,158.25,1.19,01051
...,...,...,...
395,62.40,0.53,16073
396,122.40,0.69,16074
397,94.90,0.49,16075
398,32.26,0.49,16076


### Test Merge

In [42]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(eauto_raw, on='AGS-5', how='left')
#marktstd

In [43]:
#marktstd.head().isna().sum()

## Neu entstandener Wohnraum und Anteil erneuerbarer Heiz-Energie  
Gebietsstand zum 31.12.2021  
Tabelle: Deutschlandatlas_KRS1221.csv   
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html  
   
**Wohnen_EZFH:** Fertiggestellte Wohnungen in neuen Ein-und Zweifamilienhäusern je 10.000 Einwohner in 2021   
**Wohnen_MFH:** Fertiggestellte Wohnungen in neuen Mehrfamilienhäusern je 10.000 Einwohner in 2021   
**Heizung_Erneuerbare_%:** Anteil fertiggestellter Wohnungen mit primär erneuerbarer Heizenergie an allen errichteten Wohnungen in 2021 in %    

In [44]:
# Datensatz einlesen
columns = ["Gebietskennziffer", "name", "wohn_EZFH", "wohn_MFH", "heiz_wohn"]
wohen_raw = pd.read_csv("../input/Deutschlandatlas/Deutschlandatlas_KRS1221.csv", usecols=columns, encoding="latin1", 
                             dtype={"Gebietskennziffer" : str}, sep=";", decimal=",", low_memory=False)
wohen_raw = wohen_raw.rename(columns={'wohn_EZFH': 'Wohnen_EZFH', 'wohn_MFH': 'Wohnen_MFH', 'heiz_wohn': 'Heizung_Erneuerbare_%', 'name' : 'Ort'})
wohen_raw

,Gebietskennziffer,Ort,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%
0,1001000,"Flensburg, Stadt",2.446238,31.4,0.3
1,1002000,"Kiel, Landeshauptstadt",2.027567,21.0,7.9
2,1003000,"Lübeck, Hansestadt",8.987890,32.1,24.9
3,1004000,"Neumünster, Stadt",8.259809,15.9,7.3
4,1051000,Dithmarschen,26.716497,25.3,37.0
...,...,...,...,...,...
395,16073000,Saalfeld-Rudolstadt,5.874348,4.8,25.7
396,16074000,Saale-Holzland-Kreis,9.056221,4.6,48.7
397,16075000,Saale-Orla-Kreis,7.157926,4.4,34.8
398,16076000,Greiz,6.724045,2.5,79.8


### Data Cleaning

In [45]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
wohen_raw['Gebietskennziffer'] = wohen_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
wohen_raw['AGS-5'] = wohen_raw['Gebietskennziffer'].str[:-3]
wohen_raw
# Quelle: ChatGPT (siehe Protokoll ChatGPT, Seite 3)

,Gebietskennziffer,Ort,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%,AGS-5
0,01001000,"Flensburg, Stadt",2.446238,31.4,0.3,01001
1,01002000,"Kiel, Landeshauptstadt",2.027567,21.0,7.9,01002
2,01003000,"Lübeck, Hansestadt",8.987890,32.1,24.9,01003
3,01004000,"Neumünster, Stadt",8.259809,15.9,7.3,01004
4,01051000,Dithmarschen,26.716497,25.3,37.0,01051
...,...,...,...,...,...,...
395,16073000,Saalfeld-Rudolstadt,5.874348,4.8,25.7,16073
396,16074000,Saale-Holzland-Kreis,9.056221,4.6,48.7,16074
397,16075000,Saale-Orla-Kreis,7.157926,4.4,34.8,16075
398,16076000,Greiz,6.724045,2.5,79.8,16076


In [46]:
# Spalten löschen
drop_cols = ['Gebietskennziffer', 'Ort']            
wohen_raw = wohen_raw.drop(drop_cols, axis=1)
wohen_raw

,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%,AGS-5
0,2.446238,31.4,0.3,01001
1,2.027567,21.0,7.9,01002
2,8.987890,32.1,24.9,01003
3,8.259809,15.9,7.3,01004
4,26.716497,25.3,37.0,01051
...,...,...,...,...
395,5.874348,4.8,25.7,16073
396,9.056221,4.6,48.7,16074
397,7.157926,4.4,34.8,16075
398,6.724045,2.5,79.8,16076


### Test Merge

In [47]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(wohen_raw, on='AGS-5', how='left')
#marktstd

In [48]:
#marktstd.head().isna().sum()

 # Export als SQLite Datenbank

In [49]:
# SQLite Datenbank Pfad 
SQLITE_FILE_PATH = os.path.join("..", "output", "dwh.sqlite3")

In [50]:
# Connection zur Datenbank 
con = sqlite3.connect(SQLITE_FILE_PATH)

## Tabellen exportieren

In [51]:
# Geolocation_PLZ-Tabelle anlegen
geoloc_plz.to_sql("geolocation_plz", con=con, if_exists="replace", index=False)

8160

In [52]:
# Sonnenstunden-Tabelle anlegen
sonnenstunden_plz.to_sql("sonnenstunden_plz", con=con, if_exists="replace", index=False)

8160

In [53]:
# E-Mobility-Tabelle anlegen
eauto_raw.to_sql("e-mobility", con=con, if_exists="replace", index=False)

400

In [54]:
# Erneuerbare_Energien-Tabelle anlegen
wohen_raw.to_sql("erneuerbare_energien", con=con, if_exists="replace", index=False)

400

In [55]:
# Entwicklung_PV-Tabelle anlegen
pv_anteil_raw.to_sql("entwicklung_pv", con=con, if_exists="replace", index=False)

21

In [56]:
# Strompreis-Tabelle anlegen
strompreis.to_sql("strompreis", con=con, if_exists="replace", index=False)

24

In [57]:
# Einspeiseverguetung-Tabelle anlegen
einspeisung.to_sql("einspeiseverguetung", con=con, if_exists="replace", index=False)

288

In [58]:
# EEG-Umlage-Tabelle anlegen
eeg_umlage_raw.to_sql("eeg_umlage", con=con, if_exists="replace", index=False)

288

In [59]:
# PV-Anschaffungskosten-Tabelle anlegen
pv_kosten_raw.to_sql("pv_kosten", con=con, if_exists="replace", index=False)

288

In [60]:
# Ereignisse-Tabelle anlegen
ereignisse_raw.to_sql("ereignisse", con=con, if_exists="replace", index=False)

288

## Joins

In [61]:
# Join marktstammdaten <> sonnenstunden_plz
query_15 = """  
SELECT *
FROM marktstammdaten LEFT JOIN sonnenstunden_plz
ON marktstammdaten.Postleitzahl = sonnenstunden_plz.Postleitzahl
LIMIT 5
"""
#sonnenstunden = pd.read_sql_query(query_15, con=con)
#sonnenstunden

In [62]:
# Join marktstammdaten <> e-mobility
query_16 = """  
SELECT *
FROM marktstammdaten LEFT JOIN [e-mobility]
ON marktstammdaten.[AGS-5] = [e-mobility].[AGS-5]
LIMIT 5
"""
#mobility = pd.read_sql_query(query_16, con=con)
#mobility

In [63]:
# Join marktstammdaten <> erneuerbare_energien
query_17 = """  
SELECT *
FROM marktstammdaten LEFT JOIN erneuerbare_energien
ON marktstammdaten.[AGS-5] = erneuerbare_energien.[AGS-5]
LIMIT 5
"""
#erneuerbare = pd.read_sql_query(query_17, con=con)
#erneuerbare

In [64]:
# Join marktstammdaten <> strompreis
query_18 = """  
SELECT *
FROM marktstammdaten LEFT JOIN strompreis
ON strftime('%Y', marktstammdaten.[Registrierungsdatum]) = strompreis.[Jahr]
LIMIT 5
"""
#strompreis = pd.read_sql_query(query_18, con=con)
#strompreis

In [65]:
# Join marktstammdaten <> einspeiseverguetung
# Quelle: ChatGPT (Siehe Protokoll ChatGPT, Seite 11)
query_19 = """  
SELECT *
FROM marktstammdaten LEFT JOIN einspeiseverguetung
ON strftime('%Y-%m', marktstammdaten.[Registrierungsdatum]) = strftime('%Y-%m', einspeiseverguetung.[Datum])
LIMIT 5
"""
#einspeiseverguetung = pd.read_sql_query(query_19, con=con)
#einspeiseverguetung

# Zusammenführen aller gewonnenen Daten in einer Tabelle
Für eine spätere Verwendung werden alle in der Datenbank abgelegten Tabellen mit der zentralen Marktstammdaten-Tabelle zu einer Tabelle 'marktstd_full' zusammengeführt und als csv im Output-Ordner abgelegt.   

In [66]:
# Faktentabelle: Marktstammdaten
marktstammdaten.head(5)

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011


In [67]:
# SQLite Datenbank Pfad 
SQLITE_FILE_PATH = os.path.join("..", "output", "dwh.sqlite3")

In [68]:
# Connection zur Datenbank 
con = sqlite3.connect(SQLITE_FILE_PATH)
cursor = con.cursor()

In [69]:
# Joins
marktstd_full_clean = """  
SELECT *
FROM marktstammdaten 

LEFT JOIN bevoelkerungsdichte
ON marktstammdaten.[AGS-5] = bevoelkerungsdichte.[AGS-5]

LEFT JOIN bevoelkerungsentwicklung
ON marktstammdaten.[AGS-8] = bevoelkerungsentwicklung.[AGS-8]

LEFT JOIN eigenheimquote
ON marktstammdaten.[AGS-8] = eigenheimquote.[AGS-8]

LEFT JOIN gini_index
ON marktstammdaten.Bundesland = gini_index.Bundesland

LEFT JOIN einkommensentwicklung
ON marktstammdaten.[AGS-5] = einkommensentwicklung.[AGS-5]

LEFT JOIN bip
ON marktstammdaten.[AGS-5] = bip.[AGS-5]

LEFT JOIN arbeitslosenquote
ON marktstammdaten.Bundesland = arbeitslosenquote.Bundesland

LEFT JOIN altersstruktur
ON marktstammdaten.[AGS-8]= altersstruktur.[AGS-8]

LEFT JOIN migrationshintergrund
ON marktstammdaten.[AGS-8]= migrationshintergrund.[AGS-8]

LEFT JOIN familienstand
ON marktstammdaten.[AGS-8]= familienstand.[AGS-8]

LEFT JOIN bildung
ON marktstammdaten.[AGS-5]= bildung.[AGS-5]

LEFT JOIN auslaenderanteil
ON marktstammdaten.[AGS-5]= auslaenderanteil.[AGS-5]

LEFT JOIN kriminalitaet
ON marktstammdaten.[AGS-5]= kriminalitaet.[AGS-5]

LEFT JOIN bundestagswahlergenisse
ON marktstammdaten.[AGS-8]= bundestagswahlergenisse.[AGS-8]

LEFT JOIN sonnenstunden_plz
ON marktstammdaten.Postleitzahl = sonnenstunden_plz.Postleitzahl

LEFT JOIN [e-mobility]
ON marktstammdaten.[AGS-5] = [e-mobility].[AGS-5]

LEFT JOIN erneuerbare_energien
ON marktstammdaten.[AGS-5] = erneuerbare_energien.[AGS-5]

LEFT JOIN strompreis
ON strftime('%Y', marktstammdaten.[Registrierungsdatum]) = strompreis.[Jahr]

LEFT JOIN einspeiseverguetung
ON strftime('%Y-%m', marktstammdaten.[Registrierungsdatum]) = strftime('%Y-%m', einspeiseverguetung.[Datum])
"""

# EXPLAIN-Anweisung
explain_query = f"EXPLAIN QUERY PLAN {marktstd_full_clean}"
cursor.execute(explain_query)

explain_result = cursor.fetchall()
for row in explain_result:
    print(row)

(21, 0, 0, 'SCAN marktstammdaten')
(37, 0, 0, 'SEARCH bevoelkerungsdichte USING AUTOMATIC COVERING INDEX (AGS-5=?) LEFT-JOIN')
(54, 0, 0, 'SEARCH bevoelkerungsentwicklung USING AUTOMATIC COVERING INDEX (AGS-8=?) LEFT-JOIN')
(71, 0, 0, 'SEARCH eigenheimquote USING AUTOMATIC COVERING INDEX (AGS-8=?) LEFT-JOIN')
(88, 0, 0, 'SEARCH gini_index USING AUTOMATIC COVERING INDEX (Bundesland=?) LEFT-JOIN')
(129, 0, 0, 'SEARCH einkommensentwicklung USING AUTOMATIC COVERING INDEX (AGS-5=?) LEFT-JOIN')
(146, 0, 0, 'SEARCH bip USING AUTOMATIC COVERING INDEX (AGS-5=?) LEFT-JOIN')
(163, 0, 0, 'SEARCH arbeitslosenquote USING AUTOMATIC COVERING INDEX (Bundesland=?) LEFT-JOIN')
(200, 0, 0, 'SEARCH altersstruktur USING AUTOMATIC COVERING INDEX (AGS-8=?) LEFT-JOIN')
(235, 0, 0, 'SEARCH migrationshintergrund USING AUTOMATIC COVERING INDEX (AGS-8=?) LEFT-JOIN')
(260, 0, 0, 'SEARCH familienstand USING AUTOMATIC COVERING INDEX (AGS-8=?) LEFT-JOIN')
(283, 0, 0, 'SEARCH bildung USING AUTOMATIC COVERING INDEX (AGS

In [70]:
marktstd_full_clean = pd.read_sql_query(marktstd_full_clean, con=con)

In [71]:
pd.set_option('display.max_columns', 130)
marktstd_full_clean

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Bruttoleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr,AGS-5,Fläche km2,Bevölkerung,Bevölkerung pro km2,AGS-8,Bev_Entwicklung_%,Eigentum_%,AGS-8,Bundesland,Gini_Index,AGS-5,haushaltseink_1995,haushaltseink_1996,haushaltseink_1997,haushaltseink_1998,haushaltseink_1999,haushaltseink_2000,haushaltseink_2001,haushaltseink_2002,haushaltseink_2003,haushaltseink_2004,haushaltseink_2005,haushaltseink_2006,haushaltseink_2007,haushaltseink_2008,haushaltseink_2009,haushaltseink_2010,haushaltseink_2011,haushaltseink_2012,haushaltseink_2013,haushaltseink_2014,haushaltseink_2015,haushaltseink_2016,haushaltseink_2017,haushaltseink_2018,haushaltseink_2019,haushaltseink_2020,BIP,AGS-5,...,AGS-8,Unter 3 Jahre,3 bis 5 Jahre,6 bis 9 Jahre,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,40 bis 59 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter,AGS-8,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent,AGS-8,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl,AGS-5,Auslaender_%,AGS-5,Straftaten,Einbruch,AGS-5,AGS-8,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent,Postleitzahl,Sonnenstunden,eLade,eAuto,AGS-5,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%,AGS-5,Jahr,Strom_Verbraucherpreis,Datum,Einspeiseverguetung_kleine_PV_Anlagen
0,2020-02-20 16:28:35.250812,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01 00:00:00,2007-07-20 00:00:00,In Betrieb,3.960,0.0,Volleinspeisung,4.00,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007,05515,303.28,317713.0,1048.0,05515000,0.38,51.8,05515000,Nordrhein-Westfalen,0.31,05515,16301,16293,16383,16326,16618,16707,17099,17135,17913,18648,19043,19539,19646,20484,20004,20658,20737,20711,20358,20874,21857,21689,22541,23049,23288,23306,76.27,05515,...,05515000,2.6,2.4,3.2,5.2,2.5,10.9,23.8,27.7,6.0,7.4,8.3,05515000,2.46,2.83,5.66,1.49,7.96,5.29,7.15,13.25,20.40,79.60,05515000,50.650002,37.810001,6.23,5.31,0.00,24.80,54.83,13.32,5.11,05515,11.10,05515,8316.29,87.23,05515,05515000,26.163496,24.090222,2.680048,6.676054,3.624261,32.340818,0.000000,0.494636,1.169001,0.0,0.0,0.000000,48147,1582.4,54.77,1.88,05515,7.553658,33.4,17.7,05515,2019,30.46,2019-02-01 00:00:00,11.19
1,2021-07-15 07:54:37.863709,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31 00:00:00,2013-01-31 00:00:00,In Betrieb,7.410,0.0,Teileinspeisung (einschließlich Eigenverbrauch),8.30,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013,08136,1511.39,315009.0,208.0,08136065,0.33,57.1,08136065,Baden-Württemberg,0.29,08136,14456,14527,14839,15084,15478,15721,16583,16635,17103,17473,17979,18657,19213,197

In [72]:
# durch Joins mehrfach enthaltene Spalten löschen
duplicates = marktstd_full_clean.columns.duplicated(keep='first')
marktstd_full_clean = marktstd_full_clean.loc[:, ~duplicates]
#marktstd_full_clean.head()

In [73]:
marktstd_full_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173010 entries, 0 to 3173009
Columns: 115 entries, DatumLetzteAktualisierung to Einspeiseverguetung_kleine_PV_Anlagen
dtypes: float64(65), int64(25), object(25)
memory usage: 2.7+ GB


In [74]:
# nicht benötigte Spalten löschen
drop_cols = ['Jahr','Datum']            
marktstd_full_clean = marktstd_full_clean.drop(drop_cols, axis=1)
marktstd_full_clean.head(5)

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Bruttoleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr,Fläche km2,Bevölkerung,Bevölkerung pro km2,Bev_Entwicklung_%,Eigentum_%,Gini_Index,haushaltseink_1995,haushaltseink_1996,haushaltseink_1997,haushaltseink_1998,haushaltseink_1999,haushaltseink_2000,haushaltseink_2001,haushaltseink_2002,haushaltseink_2003,haushaltseink_2004,haushaltseink_2005,haushaltseink_2006,haushaltseink_2007,haushaltseink_2008,haushaltseink_2009,haushaltseink_2010,haushaltseink_2011,haushaltseink_2012,haushaltseink_2013,haushaltseink_2014,haushaltseink_2015,haushaltseink_2016,haushaltseink_2017,haushaltseink_2018,haushaltseink_2019,haushaltseink_2020,BIP,Arbeitslosenquote_%,Unter 3 Jahre,3 bis 5 Jahre,6 bis 9 Jahre,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,40 bis 59 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl,Auslaender_%,Straftaten,Einbruch,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent,Sonnenstunden,eLade,eAuto,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%,Strom_Verbraucherpreis,Einspeiseverguetung_kleine_PV_Anlagen
0,2020-02-20 16:28:35.250812,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01 00:00:00,2007-07-20 00:00:00,In Betrieb,3.96,0.0,Volleinspeisung,4.0,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007,303.28,317713.0,1048.0,0.38,51.8,0.31,16301,16293,16383,16326,16618,16707,17099,17135,17913,18648,19043,19539,19646,20484,20004,20658,20737,20711,20358,20874,21857,21689,22541,23049,23288,23306,76.27,6.8,2.6,2.4,3.2,5.2,2.5,10.9,23.8,27.7,6.0,7.4,8.3,2.46,2.83,5.66,1.49,7.96,5.29,7.15,13.25,20.40,79.60,50.650002,37.810001,6.23,5.31,0.00,24.80,54.83,13.32,5.11,11.10,8316.29,87.23,26.163496,24.090222,2.680048,6.676054,3.624261,32.340818,0.000000,0.494636,1.169001,0.0,0.0,0.000000,1582.4,54.77,1.88,7.553658,33.4,17.7,30.46,11.19
1,2021-07-15 07:54:37.863709,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31 00:00:00,2013-01-31 00:00:00,In Betrieb,7.41,0.0,Teileinspeisung (einschließlich Eigenverbrauch),8.3,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013,1511.39,315009.0,208.0,0.33,57.1,0.29,14456,14527,14839,15084,15478,15721,16583,16635,17103,17473,17979,18657,19213,19708,19325,19952,20878,21520,21920,22706,22772,23336,24031,25288,25642,25425,82.54,3.5,2.4,2.7,3.8,6.4,3.6,7.7,17.6,29.1,7.6,9.1,10.1,6.38,2.66,9.27,2.86,13.90,9.06,12.13,22.94,35.09,64.93,39.810001,46.509998,6.18,7.49,0.01,15.81,66.12,12.91,6.11,11.65,3225.32,27.04,31.061302,26.112497,10.229063,10.494881,3.4

In [75]:
marktstd_full_clean.describe()

,NetzbetreiberpruefungStatus,Bruttoleistung,FernsteuerbarkeitNb,ZugeordneteWirkleistungWechselrichter,AnzahlModule,EinheitlicheAusrichtungUndNeigungswinkel,Inbetriebnahmejahr,Fläche km2,Bevölkerung,Bevölkerung pro km2,Bev_Entwicklung_%,Eigentum_%,Gini_Index,haushaltseink_2000,haushaltseink_2001,haushaltseink_2002,haushaltseink_2003,haushaltseink_2004,haushaltseink_2005,haushaltseink_2006,haushaltseink_2007,haushaltseink_2008,haushaltseink_2009,haushaltseink_2010,haushaltseink_2011,haushaltseink_2012,haushaltseink_2013,haushaltseink_2014,haushaltseink_2015,haushaltseink_2016,haushaltseink_2017,haushaltseink_2018,haushaltseink_2019,haushaltseink_2020,BIP,Arbeitslosenquote_%,Unter 3 Jahre,3 bis 5 Jahre,6 bis 9 Jahre,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,40 bis 59 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl,Auslaender_%,Straftaten,Einbruch,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent,Sonnenstunden,eLade,eAuto,Wohnen_EZFH,Wohnen_MFH,Heizung_Erneuerbare_%,Strom_Verbraucherpreis,Einspeiseverguetung_kleine_PV_Anlagen
count,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06
mean,7.445441e-01,2.471550e+01,1.455019e-01,9.951101e+01,1.094838e+02,7.861189e-01,2.015487e+03,1.109320e+03,2.687546e+05,4.024919e+02,1.795424e-01,7.144650e+01,2.949164e-01,1.560361e+04,1.637068e+04,1.630907e+04,1.671021e+04,1.707703e+04,1.737504e+04,1.784267e+04,1.822397e+04,1.871295e+04,1.858394e+04,1.913536e+04,1.987390e+04,2.035976e+04,2.060741e+04,2.105456e+04,2.148124e+04,2.202588e+04,2.269791e+04,2.358132e+04,2.402211e+04,2.407936e+04,7.169743e+01,4.720437e+00,2.430669e+00,2.567449e+00,3.764006e+00,6.491416e+00,3.327688e+00,6.930798e+00,1.674522e+01,3.193367e+01,7.535036e+00,9.195707e+00,9.080787e+00,2.689313e+00,2.359003e+00,5.185581e+00,1.646744e+00,7.459289e+00,5.184820e+00,6.981060e+00,1.268643e+01,1.968466e+01,8.031509e+01,3.894644e+01,4.781931e+01,6.185106e+00,7.034091e+00,1.500013e-02,1.312362e+01,6.593169e+01,1.263597e+01,5.986554e+00,1.118319e+01,4.548959e+03,4.719841e+

In [76]:
pd.set_option('display.max_rows', 130)
marktstd_full_clean.isna().sum()

DatumLetzteAktualisierung                                        0
NetzbetreiberpruefungStatus                                      0
AnlagenbetreiberMastrNummer                                      0
Bundesland                                                       0
Landkreis                                                        0
Gemeinde                                                         0
AGS-8                                                            0
Postleitzahl                                                     0
Ort                                                              0
Registrierungsdatum                                              0
Inbetriebnahmedatum                                              0
EinheitBetriebsstatus                                            0
Bruttoleistung                                                   0
FernsteuerbarkeitNb                                              0
Einspeisungsart                                               

Der Datensatz enthält vollständig bereinigte und gejointe Daten. Es gibt keine fehlenden Werte.  
Der so entstandene Datensatz wird im Output-Ordner als **marktstd_full.csv** gespeichert.   

In [77]:
output_pfad = os.path.join('../output/', 'marktstd_full.csv')

In [78]:
# Bereinigten Datensatz als csv-Datei speichern
marktstd_full = marktstd_full_clean.copy()
marktstd_full.to_csv(output_pfad, index=False, mode='w')

In [79]:
# Verbindung zur Datenbank trennen
con.close()